In [154]:
import pandas as pd
import requests
from config import api_key
import json

In [155]:
rep_candidate=pd.read_csv("rep_candidate.csv")
new_gop_finance=pd.read_csv("new_gop_finance.csv")

In [156]:
rep_candidate=rep_candidate.rename(columns={
    "state":"candidate_state"
})
rep_finance=new_gop_finance.rename(columns={
    "state":"donor_state"
})

rep_finance.to_csv("delete.csv")

In [158]:
rep_data=pd.DataFrame.merge(rep_candidate, rep_finance, on="candidate_id")

In [159]:
rep_data=rep_data.rename(columns={
    "state":"donor_state"
})
rep_data

,Unnamed: 0_x,name,candidate_id,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,party.1,candidate_status,Unnamed: 0_y,donor_state,donation_count,total_raised,cycle
0,0,"ACKISON, MELISSA",S8OH00094,OH,REP,Challenger,Senate,[2018],[2018],REP,C,0,AZ,1,500.00,2018
1,0,"ACKISON, MELISSA",S8OH00094,OH,REP,Challenger,Senate,[2018],[2018],REP,C,1,OH,88,57658.02,2018
2,0,"ACKISON, MELISSA",S8OH00094,OH,REP,Challenger,Senate,[2018],[2018],REP,C,2,OT,82,13050.30,2018
3,2,"ANDERSON, BOB",S8MN00594,MN,REP,Challenger,Senate,[2018],[2018],REP,C,3,OT,1,250.00,2018
4,3,"ANDERSON, CONSUELO E",S8HI00194,HI,REP,Challenger,Senate,[2018],[2018],REP,N,4,CA,1,1000.00,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954,19,"NEWBERGER, JAMES WALLACE",S8MN00552,MN,REP,Challenger,Senate,"[2018, 2020]",[2018],REP,P,1040,FL,1,250.00,2018
1955,19,"NEWBERGER, JAMES WALLACE",S8MN00552,MN,REP,Challenger,Senate,"[2018, 2020]",[2018],REP,P,1041,IL,1,250.00,2018
1956,19,"NEWBERGER, JAMES WALLACE",S8MN00552,MN,REP,Challenger,Senate,"[2018, 2020]",[2018],REP,P,1042,KY,1,100.00,2018
1957,19,"NEWBERGER, JAMES WALLACE",S8MN00552,MN,REP,Challenger,Senate,"[2018, 2020]",[2018],REP,P,1043,MN,278,127402.07,2018


In [160]:
# create data frame with totals fundraised by every candidate in every state they fundraised

rep_data=rep_data.filter(["name","candidate_id","candidate_state","party","incumbent_challenge_full","office_full","cycles","election_years",
                        "candidate_status","donor_state","donation_count","total_raised"])
rep_data.to_csv("rep_delete.csv")

In [161]:
# if candidate_state = donor_state, in_state=True, if not, then False

in_state=(rep_data.candidate_state==rep_data.donor_state)
in_state=in_state.to_frame()
in_state=in_state.rename(columns={
    0: "in_state"
})
in_state

,in_state
0,False
1,True
2,False
3,False
4,False
...,...
1954,False
1955,False
1956,False
1957,True


In [162]:
new_rep_data=pd.DataFrame.merge(rep_data,in_state, left_index=True, right_index=True)

new_rep_data.set_index("candidate_id")
new_rep_data.to_csv("rep_totals_by_state.csv")

In [152]:
new_rep_data["candidate_id"].nunique()
rep_aggregate=new_rep_data.groupby(["candidate_id", "in_state"]).agg({"donation_count":["sum"], "total_raised":["sum"]})
rep_aggregate


rep_aggregate.columns = rep_aggregate.columns.droplevel(1)

rep_aggregate=rep_aggregate.reset_index()

in_state=rep_aggregate.loc[rep_aggregate["in_state"]==True]
in_state
out_of_state=rep_aggregate.loc[rep_aggregate["in_state"]==False]
out_of_state

state_agg=pd.DataFrame.merge(in_state, out_of_state, on="candidate_id", suffixes=("_in_state","_out_of_state"), how="outer")
state_agg.to_csv("rep_finance_data.csv")
state_agg


,candidate_id,in_state_in_state,donation_count_in_state,total_raised_in_state,in_state_out_of_state,donation_count_out_of_state,total_raised_out_of_state
0,H6KS01179,True,409.0,391671.03,False,71.0,66044.07
1,S0WA00258,True,50.0,37253.50,False,2.0,2000.00
2,S2AZ00141,True,938.0,896980.00,False,1246.0,1969834.01
3,S2IN00109,True,1.0,500.00,NaN,NaN,NaN
4,S2NE00094,True,1895.0,687912.19,False,1705.0,1568064.45
...,...,...,...,...,...,...,...
82,S8MN00594,NaN,NaN,NaN,False,1.0,250.00
83,S8NJ00012,NaN,NaN,NaN,False,1.0,1000.00
84,S8NJ00541,NaN,NaN,NaN,False,3.0,6400.00
85,S8PA00288,NaN,NaN,NaN,False,6.0,1940.00
